In [1]:
import os
%pwd

'd:\\courses_files\\Projects\\Kidney-Disease-Classification-with-MLFlow-and-DVC\\research'

In [2]:
os.chdir('..')
%pwd

'd:\\courses_files\\Projects\\Kidney-Disease-Classification-with-MLFlow-and-DVC'

In [5]:
import dagshub

dagshub.init(
    repo_owner="namant",
    repo_name="Kidney-Disease-Classification-with-MLFlow-and-DVC",
    mlflow=True,
)

# import mlflow

# with mlflow.start_run():
#     mlflow.log_param("parameter name", "value")
#     mlflow.log_metric("metric name", 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\courses_files\Projects\Kidney-Disease-Classification-with-MLFlow-and-DVC\env\Lib\site-packages\rich\live.py:231:
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=167b8c9e-e40d-4774-830a-3e105ef5a53e&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=35d4cd73341bd81bd857f6bba8cfde3020bdf2476b48392c720f71e8ab28c190




Accessing as namant

Initialized MLflow to track repo "namant/Kidney-Disease-Classification-with-MLFlow-and-DVC"

Repository namant/Kidney-Disease-Classification-with-MLFlow-and-DVC initialized!

In [6]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts\\training\model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConifigurationManaer:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts\\training\\model.h5",
            training_data="artifacts\data_ingestion\kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/namant/Kidney-Disease-Classification-with-MLFlow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.20)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run() as run:
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("loss", self.score[0])
            mlflow.log_metric("accuracy", self.score[1])

            if tracking_uri_type_store != "file":
                mlflow.keras.log_model(
                    self.model, "model", registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")

In [16]:
try:
    config = ConifigurationManaer()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-28 17:03:32,642: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-28 17:03:32,644: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-28 17:03:32,646: INFO: common: Created directory: artifacts]
Found 93 images belonging to 2 classes.
3/3 [==============================] - 15s 4s/step - loss: 5.5865 - accuracy: 0.4839


2024/10/28 17:06:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-10-28 17:06:06,950: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\naman\AppData\Local\Temp\tmp1v79wnsu\model\data\model\assets
[2024-10-28 17:06:08,236: INFO: builder_impl: Assets written to: C:\Users\naman\AppData\Local\Temp\tmp1v79wnsu\model\data\model\assets]


d:\courses_files\Projects\Kidney-Disease-Classification-with-MLFlow-and-DVC\env\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/10/28 17:09:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
